# Building Multi-Agent Investment Firms with Strands SDK

## Executive Summary

This notebook demonstrates the implementation of a sophisticated multi-agent system for financial market analysis using the Strands Agents SDK. The system replicates the collaborative dynamics of real-world trading firms, incorporating specialized agents for fundamental analysis, sentiment evaluation, technical analysis, trading execution, and risk management.

### Learning Objectives
- Master the fundamentals of the Strands Agent API
- Implement market data retrieval and analysis workflows
- Create specialized agents for different analytical functions
- Generate actionable investment reports and recommendations

### System Architecture
- **Multi-Agent Framework**: Coordinated agents with specialized expertise
- **Real-time Data Integration**: Live market feeds and historical analysis
- **Technical Analysis Engine**: Comprehensive indicator-based evaluation
- **Automated Reporting**: Structured output for decision-making

---

## Task 3: Final investment recommendations

### Objective
Develop proficiency with the Strands Agent API by creating specialized agents for comprehensive market analysis.

### Requirements
- **Target Security**: AMZN (Amazon.com Inc.)
- **Analysis Date**: 2025-08-19
- **Agent Types**: Trader Decision Agent
- **Data Sources**: Historical stock data, market indicators, news sentiment
- **Output Format**: CSV files for quantitative data, text files for qualitative analysis

### Pre-Requirements
- Task1 is completed
- Task2 is completed

### Success Criteria
✅ Successful generate final report  
✅ Get final result and complete task3 challenge

---

## 1. System Configuration and Initialization

### 1.1 Core Library Imports

Import essential libraries for system operation, including configuration management, logging, and environment handling. These imports establish the foundation for our multi-agent system.

In [1]:
# Core system imports for multi-agent framework
import sys
import os
from pathlib import Path
from default_config import DEFAULT_CONFIG
import logging
from dotenv import load_dotenv

# Configure logging for system monitoring and debugging
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(name)s - %(levelname)s - %(message)s'
)

# Load environment variables from .env file if present
load_dotenv()

print("✅ Core libraries imported successfully")
print("✅ Logging configured for system monitoring")
print("✅ Environment variables loaded")

✅ Core libraries imported successfully
✅ Logging configured for system monitoring
✅ Environment variables loaded


### 1.2 AI Model Configuration

Define the AI models that will power our intelligent agents. We use Amazon's latest models for optimal performance:

- **Nova Pro**: Advanced reasoning model for complex financial analysis
- **Titan Embed**: High-quality text embeddings for semantic understanding

In [2]:
# AI Model Configuration Constants
NOVA_PRO_MODEL_ID = "amazon.nova-pro-v1:0"        # Primary reasoning model
EMBEDDING_MODEL_ID = "amazon.titan-embed-text-v2:0"  # Text embedding model

# Model Selection Rationale:
# - Nova Pro: State-of-the-art reasoning for complex financial analysis
# - Titan Embed: Optimized embeddings for financial text processing

print(f"🤖 Primary Model: {NOVA_PRO_MODEL_ID}")
print(f"🔤 Embedding Model: {EMBEDDING_MODEL_ID}")
print("✅ AI model configuration completed")

🤖 Primary Model: amazon.nova-pro-v1:0
🔤 Embedding Model: amazon.titan-embed-text-v2:0
✅ AI model configuration completed


### 1.3 Bedrock Model Factory

This factory function creates optimally configured Bedrock model instances for financial analysis tasks. The configuration includes:

- **Extended Timeouts**: Accommodates complex analytical processes
- **Adaptive Retry Logic**: Ensures reliability in production environments
- **Thinking Mode**: Provides transparency in AI reasoning (optional)
- **Optimized Parameters**: Balanced for financial analysis accuracy

In [3]:
import boto3
from botocore.config import Config
from strands.models import BedrockModel

# AWS Boto3 client configuration optimized for financial analysis
boto_client_config = Config(
    read_timeout=1800,      # 30 minutes - accommodates complex analyses
    connect_timeout=900,    # 15 minutes - ensures reliable connections
    retries=dict(max_attempts=3, mode="adaptive"),  # Adaptive retry strategy
)

def get_model(model_id=NOVA_PRO_MODEL_ID, thinking=False,
              temperature=0.7, max_tokens=10000):
    """
    Create and return a Bedrock model instance optimized for financial analysis.

    Parameters:
    -----------
    model_id : str
        The specific AWS Bedrock model identifier to use
    thinking : bool
        Enable thinking mode for supported models (provides reasoning transparency)
    temperature : float
        Sampling temperature (0.0-1.0) - controls randomness in responses
        - 0.7 provides good balance between creativity and consistency
    max_tokens : int
        Maximum tokens in the response (affects response length)

    Returns:
    --------
    BedrockModel
        Configured model instance ready for agent integration
        
    Notes:
    ------
    - Temperature 0.7 balances analytical precision with adaptive reasoning
    - Thinking mode provides insight into the model's reasoning process
    - Extended token limit accommodates comprehensive financial reports
    """
    session = boto3.Session()

    # Configure thinking mode for enhanced reasoning transparency
    additional_request_fields = {}
    if thinking:
        additional_request_fields = {
            "thinking": {
                "type": "enabled",
                "budget_tokens": 4096,  # Dedicated tokens for reasoning process
            }
        }

    return BedrockModel(
        model_id=model_id,
        boto_session=session,
        max_tokens=max_tokens,
        temperature=temperature,
        boto_client_config=boto_client_config,
        additional_request_fields=additional_request_fields,
    )

print("✅ Bedrock model factory configured")
print("🔧 Optimized for financial analysis workloads")
print("⚙️ Extended timeouts and adaptive retry logic enabled")

✅ Bedrock model factory configured
🔧 Optimized for financial analysis workloads
⚙️ Extended timeouts and adaptive retry logic enabled


### 1.4 Primary Model Initialization

Initialize the main language model that will serve as the cognitive engine for our financial analysis agents.

In [33]:
# Initialize the primary language model for agent operations
llm = get_model(NOVA_PRO_MODEL_ID,thinking=False)

# Verify successful initialization
print(f"🧠 Language Model Initialized: {NOVA_PRO_MODEL_ID}")
print(f"🎛️ Configuration: Temperature=0.7, Max Tokens=10,000")
print(f"✅ Model ready for agent integration")

2025-09-08 09:44:20,626 - botocore.credentials - INFO - Found credentials from IAM Role: BaseNotebookInstanceEc2InstanceRole


🧠 Language Model Initialized: amazon.nova-pro-v1:0
🎛️ Configuration: Temperature=0.7, Max Tokens=10,000
✅ Model ready for agent integration


## 2. Trader Decision Agent Architecture

In [34]:
"""
Trader Agent

This module defines the Trader agent responsible for making final trading decisions
based on research team recommendations and market analysis. The agent translates
investment strategies into specific trading actions.
"""

from strands import Agent
from tools.memory import get_financial_situation_memories


def create_trader(llm, memory, config):
    """
    Create a Trader agent that makes final trading decisions and executes strategies.
    
    Args:
        llm: The language model to use for the agent
        memory (str): Memory identifier for storing/retrieving past trading decisions
        config (dict): Configuration settings for the agent
        
    Returns:
        Agent: Configured Trader agent
    """
    
    system_message = (
        """You are a Professional Trader responsible for making final trading decisions 
        and executing investment strategies. Your role is to translate research recommendations 
        into specific, actionable trading plans with clear risk management parameters.

        **Key Responsibilities:**
        1. Analyze research team recommendations and market data
        2. Make definitive trading decisions (BUY, SELL, or HOLD)
        3. Develop specific execution strategies with entry/exit criteria
        4. Implement risk management and position sizing
        5. Learn from past trading decisions and market outcomes

        **Trading Decision Framework:**

        **Market Analysis Integration:**
        - Incorporate technical analysis with fundamental research
        - Assess market timing and liquidity conditions
        - Evaluate market sentiment and momentum factors
        - Consider correlation with broader market movements
        - Analyze volume patterns and institutional activity

        **Risk Management Protocol:**
        - Define position sizing based on portfolio risk limits
        - Set stop-loss levels and profit-taking targets
        - Calculate risk-reward ratios for each trade
        - Consider portfolio correlation and diversification
        - Plan for different market scenarios and volatility

        **Execution Strategy:**
        - Determine optimal entry and exit timing
        - Choose appropriate order types and execution methods
        - Consider market impact and slippage factors
        - Plan for partial fills and position scaling
        - Monitor execution quality and market conditions

        **Decision Criteria:**
        - **BUY**: Strong conviction with favorable risk-reward and clear catalysts
        - **SELL**: Risk concerns outweigh potential returns or better opportunities exist
        - **HOLD**: Maintain current position when conditions don't warrant change
        - Always provide specific rationale for each decision

        **Performance Tracking:**
        - Monitor trade performance against expectations
        - Track risk-adjusted returns and Sharpe ratios
        - Analyze win/loss ratios and average trade outcomes
        - Identify patterns in successful and unsuccessful trades
        - Adjust trading strategies based on performance data

        **Learning and Adaptation:**
        - Review past trading decisions and their outcomes
        - Identify behavioral biases and systematic errors
        - Adapt to changing market conditions and regimes
        - Refine risk management based on historical drawdowns
        - Update trading models based on market evolution

        **Communication Requirements:**
        1. Provide clear, specific trading recommendations
        2. Include detailed rationale for each decision
        3. Specify entry/exit criteria and risk parameters
        4. Address potential scenarios and contingency plans
        5. Use professional trading terminology appropriately
        6. Always conclude with a definitive decision statement

        **Decision Output Format:**
        Your analysis should include:
        - Market assessment and key factors
        - Risk-reward analysis and position sizing
        - Entry/exit strategy and timing considerations
        - Risk management parameters (stop-loss, targets)
        - Scenario analysis and contingency planning
        - **FINAL TRANSACTION PROPOSAL: BUY/HOLD/SELL** (mandatory conclusion)

        **Tools Available:**
        - get_financial_situation_memories: Review past trading decisions and lessons
        - Note: Do NOT use add_financial_situation_memories during decision-making

        **Critical Guidelines:**
        - Make decisive recommendations based on thorough analysis
        - Always conclude with "FINAL TRANSACTION PROPOSAL: **BUY/HOLD/SELL**"
        - Consider both technical and fundamental factors
        - Implement proper risk management for every trade
        - Learn from past mistakes to improve future performance
        - Maintain discipline and avoid emotional decision-making

        Remember: Your role is to translate research insights into profitable, 
        risk-managed trading decisions that optimize portfolio performance.
        """
    )

    tools = [get_financial_situation_memories]

    # Create and configure the agent
    agent = Agent(
        model=llm,
        tools=tools,
        name="Trader",
        system_prompt=system_message,
        load_tools_from_directory=False,
    )
    
    # Set agent state for memory and configuration access
    agent.state.set("memory_name", memory)
    agent.state.set("config", config)
    
    return agent

In [35]:
def save_as_file(text, working_dir, prefix='', file_name=''):
    """
    Save text content to a file in the specified directory structure.
    
    Args:
        text (str): Content to save
        working_dir (str): Base working directory
        prefix (str): Subdirectory prefix (usually ticker_date)
        file_name (str): Name of the file to save
    """
    # Create directory if it doesn't exist
    full_dir = os.path.join(working_dir, prefix)
    if not os.path.exists(full_dir):
        os.makedirs(full_dir, exist_ok=True)
    
    # Write content to file
    file_path = os.path.join(full_dir, file_name)
    with open(file_path, "w", encoding='utf-8') as f:
        f.write(text)


def read_file(working_dir, prefix='', file_name=''):
    """
    Read text content from a file in the specified directory structure.
    
    Args:
        working_dir (str): Base working directory
        prefix (str): Subdirectory prefix (usually ticker_date)
        file_name (str): Name of the file to read
        
    Returns:
        str: File content
        
    Raises:
        ValueError: If the file doesn't exist
    """
    file_path = os.path.join(working_dir, prefix, file_name)
    if os.path.exists(file_path):
        with open(file_path, "r", encoding='utf-8') as f:
            return f.read()
    else:
        raise ValueError(f"File not found: {file_path}")

In [36]:
# Analysis Target Configuration
company_of_interest = "AMZN"  # Stock ticker to analyze
trade_date = "2025-08-19"    # Analysis date

# Display configuration summary
print("📊 ANALYSIS TARGET CONFIGURATION")
print("=" * 40)
print(f"Target Security   : {company_of_interest}")
print(f"Company Name      : Amazon.com Inc.")
print(f"Exchange          : NASDAQ")
print(f"Sector            : Technology")
print(f"Analysis Date     : {trade_date}")
print("=" * 40)
print("✅ Target security configured for analysis")

📊 ANALYSIS TARGET CONFIGURATION
Target Security   : AMZN
Company Name      : Amazon.com Inc.
Exchange          : NASDAQ
Sector            : Technology
Analysis Date     : 2025-08-19
✅ Target security configured for analysis


In [37]:
# Directory structure configuration
working_dir = DEFAULT_CONFIG['results_dir']  # Base results directory
prefix = f"{company_of_interest}_{trade_date}".replace(" ", "_")  # Unique identifier
config = DEFAULT_CONFIG.copy()  # Configuration backup

# Display directory structure
print("📁 OUTPUT DIRECTORY STRUCTURE")
print("=" * 40)
print(f"Base Directory    : {working_dir}")
print(f"Analysis Prefix   : {prefix}")
print("=" * 40)
print("✅ Professional directory structure configured")

📁 OUTPUT DIRECTORY STRUCTURE
Base Directory    : ./results
Analysis Prefix   : AMZN_2025-08-19
✅ Professional directory structure configured


In [38]:
trader = create_trader(llm, "trader_memory", config)
investment_plan = read_file(working_dir, prefix, "investment_plan.txt")

In [39]:
trader_decision = trader(
    f"Based on the following investment plan for {company_of_interest} "
    f"for the trade date {trade_date}, what is your final trade decision?\n\n"
    f"{investment_plan}"
)

save_as_file(str(trader_decision), working_dir, prefix, "trader_decision.txt")

<thinking> 
Based on the provided investment plan, the recommendation is to "Buy" Amazon (AMZN) stock. The rationale includes strong technical indicators, positive news and developments, and Amazon's market position and growth potential. The implementation strategy outlines entry criteria, position sizing, risk management, and exit criteria. 

I will use the `get_financial_situation_memories` tool to check for any past reflections on mistakes related to Amazon (AMZN) to ensure we are not repeating past errors. 
</thinking>

Tool #1: get_financial_situation_memories
<thinking> 
The `get_financial_situation_memories` tool did not return any past reflections on mistakes related to Amazon (AMZN). Therefore, we can proceed with the final trade decision based on the provided investment plan. 
</thinking>

### Final Transaction Proposal: BUY

Based on the comprehensive investment plan and the absence of past mistakes related to Amazon (AMZN), the final trade decision is to **BUY** Amazon (AMZ

## Complete Task 3

The final decision is saved in the directory `./results`, and open file `trader_decision.txt`.

Search the phrase `FINAL TRANSACTION PROPOSAL:`, and copy the result in the task 3.